<a href="https://colab.research.google.com/github/Learnheart/URL-Classification/blob/main/Ensemble.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow==2.16.1
# pip install Keras-Preprocessing

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 17.2 MB/s eta 0:00:00
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.2.0
    Uninstalling ml-dtypes-0.2.0:
      Successfully uninstalled ml-dtypes-0.2.0
  Attempting uninstall: h5py
    Found existing installation: h5py 3.9.0
    Uninstalling h5py-3.9.0:
      Successfully uninstalled h5py-3.9.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.15.2
    Uninstalling tensorboard-2.15.2:
      Successfully uninstalled tensorboard-2.15.2
  Attempting uninstall: keras
    Fo

In [ ]:
import tensorflow
print(tensorflow.__version__)

2.16.1


In [ ]:
import pandas as pd
import numpy as np
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_selection import chi2
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from keras.utils import to_categorical
from keras.models import Sequential
import re
from keras.preprocessing import sequence
# from keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.text import text_to_word_sequence
from sklearn.linear_model import SGDClassifier
from keras.preprocessing import sequence
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.text import text_to_word_sequence
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier
from tensorflow.keras.utils import plot_model


!pip3 install pyvi
from pyvi import ViUtils
import pickle

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 51.4 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def url_to_text(url):
  # remove stopwords
  url = url.replace('.html','').replace('.htm','').replace('http://','').replace('https://','')
  url = re.sub('^(.*?/)','/', url) # remove domains
  url = re.sub('[0-9]+', '', url)
  url = re.sub('[_\-/]+', ' ', url)

  return ViUtils.remove_accents(url.lower()).decode()


In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer

stemmer = SnowballStemmer("english", ignore_stopwords=True)
class StemmedCountVectorizer(CountVectorizer):
    def build_analyzer(self):
        analyzer = super(StemmedCountVectorizer, self).build_analyzer()
        return lambda doc: ([stemmer.stem(w) for w in analyzer(doc)])

stemmed_count_vect = StemmedCountVectorizer(stop_words='english', ngram_range=(3,3))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
from collections import Counter
import joblib
from tensorflow.keras.models import load_model
# random forest
rf_model = joblib.load('/content/drive/MyDrive/NCKH/HyperModel/random_forest_origin.pkl')
# xgboost
xgboost_model = joblib.load('/content/drive/MyDrive/NCKH/HyperModel/hyper_xgboost.pkl')
# svm
svm_model = joblib.load('/content/drive/MyDrive/NCKH/HyperModel/hyper_svm(2).pkl')
# cnn
cnn_model = load_model('/content/drive/MyDrive/NCKH/HyperModel/cnn_model_16_191.h5')
# decision tree
dt_model = joblib.load('/content/drive/MyDrive/NCKH/HyperModel/decision_tree_origin.pkl')
# naive bayes
nb_model = joblib.load('/content/drive/MyDrive/NCKH/HyperModel/naive_bayes_origin.pkl')

In [ ]:
categories = ['Automotive', 'Books & Literature', 'Business & Finance',
       'Careers', 'Education', 'Entertainment & Art',
       'Family & Relationships', 'Food & Drink', 'Healthy Living',
       'Home & Garden', 'News & Politics', 'Real Estate', 'Science & Technology',
       'Sports', 'Style & Fashion', 'Travel', 'Games', 'Laws & Policies', 'Environment' ]

In [ ]:
new_input_url = input('URL ')
new_input_processed = url_to_text(new_input_url)

URL https://gamevui.vn/


#Pre-processing


In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(oov_token='<OOV>')
tokenizer.fit_on_texts(new_input_processed)
vocab_size = len(tokenizer.word_index) + 1
max_length=30

print("Vocabulary size:", vocab_size)

def process_texts(texts):
    sequences = tokenizer.texts_to_sequences(texts=texts)
    return pad_sequences(sequences, maxlen=max_length, padding='post')

Vocabulary size: 2


In [ ]:
#random forest
rf_prediction = rf_model.predict([new_input_processed])
#xgboost
xgboost_prediction = xgboost_model.predict([new_input_processed])
#svm
svm_prediction = svm_model.predict([new_input_processed])
# decision tree
dt_prediction = dt_model.predict([new_input_processed])
# naive bayes
nb_prediction = nb_model.predict([new_input_processed])
#cnn
cnn_prediction = cnn_model.predict(process_texts([new_input_processed]))

# Convert CNN prediction to category
cnn_predicted_index = np.argmax(cnn_prediction, axis=1)[0]
cnn_predicted_category = categories[cnn_predicted_index]

# Combine predictions
all_predictions = [rf_prediction[0], xgboost_prediction[0], svm_prediction[0], dt_prediction[0], nb_prediction[0], cnn_predicted_category]

# Perform hard voting
final_prediction = Counter(all_predictions).most_common(1)[0][0]
print(f"Predicted category for '{new_input_url}': {final_prediction}")

# print("Final Prediction:", final_prediction)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Predicted category for 'https://gamevui.vn/': Books & Literature
